In [1]:
import numpy as np
import pandas as pd
import os





city_hierarchy = {
    "united states": {
        "california": {
            "los angeles area": [
                "los angeles (la)", "west la", "hollywood", "w. hollywood", 'los angeles','la',
                "century", "chinatown", "studio", "westwood", "venice", 
                "los feliz", "encino", "sherman oaks", "santa monica", 
                "malibu", "beverly hills", "bel air", "manhattan beach", 
                "rancho park"
            ],
            "san francisco area": ["san francisco"],
            "other california": ["pasadena"]
        },
        "new york": {
            "new york city (nyc)": ["manhattan (new york)", "brooklyn", "queens"],
            "other new york": ["marietta"]
        },
        "georgia": {
            "atlanta": ["roswell"]
        },
        "nevada": {
            "las vegas": []
        },
        "other locations": ["st. hermosa beach",'lys velas']
    }
}

restaurant_hierarchy = {
    "american": {
        "styles": ["american (new)", "american (traditional)", "dive american",'american ( new )'],
        "barbecue": ["bbq"],
        "southern": ["cajun", "southern", "southern/soul", "southwestern"],
        "fast food & casual": [
            "hamburgers", "hot dogs", "fast food", "steak houses", 'chicken',
            "steakhouses", "health food", "vegetarian", "sandwiches",'delicatessen','delis'
        ],
        "coffee": ["coffee bar", "coffee shops", "coffee shops/diners", "coffeehouses"],
        "dining": ["diners", "cafeterias", "buffets"]
    },
    "asian": ["chinese", "japanese", "thai", "vietnamese", "indian", "indonesian"],
    "european": {
        "french": ["french", "french (classic)", "french (new)", "french bistro",'french ( new )','french ( classic )'],
        "others": [
            "greek", "greek and middle eastern", "italian", 
            "nuova cucina italian", "east european", 
            "polish", "russian", "scandinavian",'continental'
        ]
    },
    "latin american & caribbean": [
        "mexican", "tex-mex", "latin american", "mexican/latin american/spanish",
        "caribbean", "cuban", "tel caribbean"
    ],
    "mediterranean": ["mediterranean", "greek", "greek and middle eastern"],
    "seafood": ["seafood"],
    "eclectic/international": [
        "eclectic", "international", "pacific new wave", 
        "pacific rim", "californian", "old san francisco", 
        "only in las vegas", "ext 6108 international", 
        "or 212/632-5100 american"
    ],
    "desserts & specialty": ["desserts", "coffeehouses"],
    "pizza": ["pizza"]
}





# col = 'type'
# data = list(np.unique(list(np.unique(list(pd.read_csv(f'data/{task}/tableA.csv')[col])))+list(np.unique(list(pd.read_csv(f'data/{task}/tableB.csv')[col])))))



def make_mask(df,frac):
    L = len(COL)*len(df)
    L_disturb = int(np.ceil(L*frac))
    disturb_mask = np.zeros((len(df),len(COL)))
    total_elements = disturb_mask.size
    random_indices = np.random.choice(total_elements, L_disturb, replace=False)
    disturb_mask.flat[random_indices] = 1
    return disturb_mask 


def find_parents(hierarchy, target, parents=None):
    if parents is None:
        parents = []

    for key, value in hierarchy.items():
        if key == target:  # If the target itself is a key
            return parents + [key]
        if isinstance(value, dict):  # If value is a nested dictionary
            result = find_parents(value, target, parents + [key])
            if result != -1:
                return result
        elif isinstance(value, list) and target in value:  # If value is a list containing the target
            return parents + [key]

    return -1  # Return -1 if the target is not found


def disturb_func(col,value):
    if col =='city':
        
        parents = find_parents(city_hierarchy, value.lower())
        if parents == -1: print(value)

        np.random.shuffle(parents)
        new_velue = parents[0]
        

    elif col == 'typee':
        parents = find_parents(restaurant_hierarchy, value.lower())
        if parents == -1: print(value)
        np.random.shuffle(parents)
        new_velue = parents[0]

    elif col =='class':
        data = [int(value)]
        number_hierarchy = {
            "0-99": [n for n in data if 0 <= n <= 99],
            "100-199": [n for n in data if 100 <= n <= 199],
            "200-299": [n for n in data if 200 <= n <= 299],
            "300-399": [n for n in data if 300 <= n <= 399],
            "400-499": [n for n in data if 400 <= n <= 499],
            "500-599": [n for n in data if 500 <= n <= 599],
            "600-699": [n for n in data if 600 <= n <= 699],
            "700-799": [n for n in data if 700 <= n <= 799]
        }
        new_velue = next((key for key, values in number_hierarchy.items() if values), None)

    return new_velue
task = 'Fodors-Zagat'
COL = ['class','typee','city']

def column_entropy(column):
    column = column.fillna('nan')
    value_counts = column.value_counts()
    probabilities = value_counts / len(column)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy




import os
import random


for frac in [0, 0.1,0.2,0.3,0.4,0.5,0.7,0.9]:
    L =[]
    R = []

    for rep in range(1,101):
        df = pd.read_csv("/Users/mohammad/Desktop/EM_Heterogeneity/data/" + task  +'/test.csv')


        mask_left = make_mask(df,frac)
        mask_right = make_mask(df,frac)


        df['left_class'] = df['left_class'].astype(str)
        df['right_class'] = df['right_class'].astype(str)
        x = 0
        for i,col in enumerate(COL):
            
            df.loc[mask_left[:,i] == 1, 'left_'+col] = df.loc[mask_left[:,i] == 1, 'left_'+col].apply(lambda x: disturb_func(col, x))
            df.loc[mask_right[:,i] == 1, 'right_'+col] = df.loc[mask_right[:,i] == 1, 'right_'+col].apply(lambda x: disturb_func(col, x))

            x+=(column_entropy(df['left_'+col]) +column_entropy(df['right_'+col]))

        L.append(x)
    if frac ==0:
        base = np.average(L).copy()
    base =1
    print(round(np.average(L)/base,2), round(np.sqrt(np.std(L))/base,2))

print('Fodor')


26.99 0.0
27.74 0.39
27.66 0.43
27.16 0.46
26.34 0.47
25.19 0.48
22.38 0.47
18.5 0.42
Fodor


In [32]:
import numpy as np
import pandas as pd
import os





city_hierarchy = {
    "united states": {
        "california": {
            "los angeles area": [
                "los angeles (la)", "west la", "hollywood", "w. hollywood", 'los angeles','la',
                "century", "chinatown", "studio", "westwood", "venice", 
                "los feliz", "encino", "sherman oaks", "santa monica", 
                "malibu", "beverly hills", "bel air", "manhattan beach", 
                "rancho park"
            ],
            "san francisco area": ["san francisco"],
            "other california": ["pasadena"]
        },
        "new york": {
            "new york city (nyc)": ["manhattan (new york)", "brooklyn", "queens"],
            "other new york": ["marietta"]
        },
        "georgia": {
            "atlanta": ["roswell"]
        },
        "nevada": {
            "las vegas": []
        },
        "other locations": ["st. hermosa beach",'lys velas']
    }
}

restaurant_hierarchy = {
    "american": {
        "styles": ["american (new)", "american (traditional)", "dive american",'american ( new )'],
        "barbecue": ["bbq"],
        "southern": ["cajun", "southern", "southern/soul", "southwestern"],
        "fast food & casual": [
            "hamburgers", "hot dogs", "fast food", "steak houses", 'chicken',
            "steakhouses", "health food", "vegetarian", "sandwiches",'delicatessen','delis'
        ],
        "coffee": ["coffee bar", "coffee shops", "coffee shops/diners", "coffeehouses"],
        "dining": ["diners", "cafeterias", "buffets"]
    },
    "asian": ["chinese", "japanese", "thai", "vietnamese", "indian", "indonesian"],
    "european": {
        "french": ["french", "french (classic)", "french (new)", "french bistro",'french ( new )','french ( classic )'],
        "others": [
            "greek", "greek and middle eastern", "italian", 
            "nuova cucina italian", "east european", 
            "polish", "russian", "scandinavian",'continental'
        ]
    },
    "latin american & caribbean": [
        "mexican", "tex-mex", "latin american", "mexican/latin american/spanish",
        "caribbean", "cuban", "tel caribbean"
    ],
    "mediterranean": ["mediterranean", "greek", "greek and middle eastern"],
    "seafood": ["seafood"],
    "eclectic/international": [
        "eclectic", "international", "pacific new wave", 
        "pacific rim", "californian", "old san francisco", 
        "only in las vegas", "ext 6108 international", 
        "or 212/632-5100 american"
    ],
    "desserts & specialty": ["desserts", "coffeehouses"],
    "pizza": ["pizza"]
}





# col = 'type'
# data = list(np.unique(list(np.unique(list(pd.read_csv(f'data/{task}/tableA.csv')[col])))+list(np.unique(list(pd.read_csv(f'data/{task}/tableB.csv')[col])))))



def make_mask(df,frac):
    L = len(COL)*len(df)
    L_disturb = int(np.ceil(L*frac))
    disturb_mask = np.zeros((len(df),len(COL)))
    total_elements = disturb_mask.size
    random_indices = np.random.choice(total_elements, L_disturb, replace=False)
    disturb_mask.flat[random_indices] = 1
    return disturb_mask 


def find_parents(hierarchy, target, parents=None):
    if parents is None:
        parents = []

    for key, value in hierarchy.items():
        if key == target:  # If the target itself is a key
            return parents + [key]
        if isinstance(value, dict):  # If value is a nested dictionary
            result = find_parents(value, target, parents + [key])
            if result != -1:
                return result
        elif isinstance(value, list) and target in value:  # If value is a list containing the target
            return parents + [key]

    return -1  # Return -1 if the target is not found


def disturb_func(col,value):
    if col =='city':
        
        parents = find_parents(city_hierarchy, value.lower())
        if parents == -1: print(value)

        np.random.shuffle(parents)
        new_velue = parents[0]
        

    elif col == 'typee':
        parents = find_parents(restaurant_hierarchy, value.lower())
        if parents == -1: print(value)
        np.random.shuffle(parents)
        new_velue = parents[0]

    elif col =='class':
        data = [int(value)]
        number_hierarchy = {
            "0-99": [n for n in data if 0 <= n <= 99],
            "100-199": [n for n in data if 100 <= n <= 199],
            "200-299": [n for n in data if 200 <= n <= 299],
            "300-399": [n for n in data if 300 <= n <= 399],
            "400-499": [n for n in data if 400 <= n <= 499],
            "500-599": [n for n in data if 500 <= n <= 599],
            "600-699": [n for n in data if 600 <= n <= 699],
            "700-799": [n for n in data if 700 <= n <= 799]
        }
        new_velue = next((key for key, values in number_hierarchy.items() if values), None)

    return new_velue
task = 'Fodors-Zagat'
COL = ['class','typee','city']

def column_entropy(column):
    column = column.fillna('nan')
    value_counts = column.value_counts()
    probabilities = value_counts / len(column)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy




import os
import random


for frac in [0, 0.1,0.2,0.3,0.4,0.5]:
    L =[]
    R = []

    for rep in range(1,101):
        df = pd.read_csv("/Users/mohammad/Desktop/EM_Heterogeneity/data/" + task  +'/test.csv')


        mask_left = make_mask(df,frac)
        mask_right = make_mask(df,frac)


        df['left_class'] = df['left_class'].astype(str)
        df['right_class'] = df['right_class'].astype(str)
        x = 0
        for i,col in enumerate(COL):
            
            df.loc[mask_left[:,i] == 1, 'left_'+col] = df.loc[mask_left[:,i] == 1, 'left_'+col].apply(lambda x: disturb_func(col, x))
            df.loc[mask_right[:,i] == 1, 'right_'+col] = df.loc[mask_right[:,i] == 1, 'right_'+col].apply(lambda x: disturb_func(col, x))

            x+=(column_entropy(df['left_'+col]) +column_entropy(df['right_'+col]))

        L.append(x)
    if frac ==0:
        base = np.average(L).copy()
    base =1
    print(round(np.average(L)/base,2), round(np.sqrt(np.std(L))/base,2))

print('Fodor')


26.99 0.0
27.77 0.38
27.66 0.44
27.14 0.48
26.3 0.48
25.25 0.5
Fodor


In [24]:

import numpy as np
import pandas as pd
import time

brand_hierarchy = {
    "Electronics": {
        "Computers": ["acer", "alienware", "apple", "asus", "dell", "lenovo", "microsoft", "a-data", "a4tech", "accell", "avid", "azio", "ibm", "4inkjets", "acme", "acme made", "acp", "acp-ep memory", "agama", "aoc", "apc", "arclyte technologies inc.", "atlona", "atrend", "aurora", "avf", "basyx", "bell o", "bling software", "blurex", "bracketron", "buffalo technology", "built ny"],
        "Components": ["3m", "corsair", "kingston", "seagate", "sandisk", "western digital", "dane", "dane-elec", "diamond multimedia", "bits limited", "coolmax", "memory upgrades", "patriot", "buffalo", "centon", "chief", "chief manufacturing", "clickfree", "cobra", "comprehensive", "comprehensive video", "coolguard", "cp tech", "cp technology", "crown", "edge", "eforcity", "ep memory", "eveready", "focus"],
        "Mobile Devices": ["htc", "jabra", "samsung", "sony", "nokia", "apple", "blackberry", "motorola", "mybat", "coby", "lg", "vtech"],
        "Audio & Video": ["audio-technica", "bose", "jvc", "pioneer", "plantronics", "sony", "audiovox", "audiosource", "axis", "axis communications", "boss", "creative", "crosley", "diamond", "jaton", "razer", "sennheiser", "sharp", "skullcandy", "creative labs", "hercules", "ihome", "elite", "matic", "clarion mobile electronics", "dj tech", "dj-tech", "fanatics", "philips", "rca", "sirius", "sirius satellite radio"],
        "Networking": ["cisco", "d-link", "linksys", "netgear", "tp-link", "zyxel", "actiontec", "trendnet", "trendware", "amp", "amped wireless", "link depot", "syba", "syba multimedia", "netopia", "u.s. robotics"],
        "Storage Devices": {
            "Internal Storage": ["seagate", "western digital", "toshiba", "crucial", "ocz", "ocz technologies", "quantum", "transcend", "edge tech"],
            "External Storage": ["sandisk", "kingston", "lacie", "buffalo", "cavalry", "buslink", "apricorn", "iomega", "pny"]
        }
    },
    "Office Supplies": {
        "Paper Products": ["hammermill", "smead", "avery", "cardinal", "pendaflex", "boise", "domtar", "pacon", "pacon products", "boise cascade", "tops business forms"],
        "Writing Instruments": ["bic", "sharpie", "pilot", "papermate", "uniball", "prismacolor", "expo", "tops", "pentel", "pentel of america ltd.", "stabilo"],
        "Organization": ["wilson jones", "officemate", "post-it", "swingline", "bankers box", "buddy products", "c-line", "smead manufacturing company", "rolodex", "globe weis", "ghent", "safco products"],
        "Other Office Supplies": ["acco", "buddy products", "dixie", "rubbermaid", "samsill", "bond street", "bond street ltd.", "elmer s", "exact", "staples", "at t", "avery personal creations", "paperpro", "redi-tag"]
    },
    "Accessories": {
        "Computer Accessories": ["accessory power", "belkin", "case logic", "kensington", "targus", "brenthaven", "cables to go", "cables unlimited", "iogear", "zagg", "otterbox", "incipio", "griffin", "vantec", "logitech", "cooler master", "coolermaster", "pluggable technologies", "siig", "planar", "planar systems", "plugable technologies"],
        "Mobile Accessories": ["zagg", "otterbox", "speck", "incipio", "griffin", "blueant", "iluv", "amzer", "just mobile", "bluelounge", "bluelounge design", "roadwired"],
        "Photography Accessories": ["lowepro", "moshi", "manfrotto", "tamron", "sigma", "ape case", "dolica", "tiffen", "tenba", "zeikos", "arkon", "arrowmounts", "bower", "platt", "platt cases", "sima", "golla", "humminbird"]
    },
    "Software": {
        "Operating Systems": ["microsoft"],
        "Creative Tools": ["adobe", "encore software", "acd", "acd systems", "avid technology", "roxio", "serif", "the learning company"],
        "Security": ["symantec", "webroot", "mcafee", "kaspersky"],
        "Utilities": ["avg", "norton", "ccleaner", "winrar", "plustek", "neosafe", "advanced system care", "iosafe"]
    },
    "Consumer Electronics": {
        "Audio": ["bose", "audio-technica", "plantronics", "skullcandy", "sony", "aztec", "soundstorm", "fellowes", "cyber acoustics", "ultimate ears"]
    },
    "Printing & Imaging": {
        "Printers": ["brother", "canon", "epson", "hp", "xerox", "oki", "lexmark", "dymo"],
        "Cameras": ["nikon", "olympus", "panasonic", "sony", "vivitar", "fujifilm", "kodak", "polaroid", "fuji", "garmin"],
        "Projectors": ["optoma", "benq", "epson", "nec", "hitachi", "elite screens", "viewsonic", "sunpak"]
    },
    "Home Appliances": {
        "Small Appliances": ["black & decker", "dyson", "kitchenaid", "cuisinart", "allied", "haier", "emerson", "magna visual", "keurig"]
    },
    "Entertainment": {
        "Characters": ["disney", "hello kitty"],
        "Media": ["sirius xm", "spotify", "pandora", "ilive", "flip video", "eye-fi", "favi entertainment", "vizio", "sirius", "sirius satellite radio"]
    },
    "Miscellaneous": {
        "General": ["generic", "unknown", "3m #", "blue crane digital", "black n  red", "green onions supply", "u.s. brown bear", "universal", "americopy", "dealz4real", "egpchecks", "add on", "sage", "sumas"],
        "Multi-Category": ["3m", "amazonbasics", "anker", "eco-products", "advantus", "acs", "casio", "ford"]
    },
    "Uncategorized": ["accessorieszone", "agf", "action", "adesso", "alama", "aluratek", "antec", "balt", "best rite", "booq", "brainydeal", "bti", "bushnell", "compatible", "digital concepts", "dreamgear", "duracell", "energizer", "evga", "ifixit", "joby", "kensmart", "lowrance", "macsense", "marware", "mivizu", "msi", "mustang", "namo", "nuance", "penpower", "quartet", "storex", "think outside", "treque", "tribeca", "tripp lite", "trodat", "v7", "vaultz", "writeright", "zalman", "clarion", "clarion software", "simplism", "cocoon", "cocoon innovations", "compucessory", "concord", "csdc", "cta", "cta digital", "curtis", "cyberpower", "da-lite", "da-lite screen", "defender", "deluxe", "diaper dude", "digipower", "digital innovations", "digital peripheral solutions", "directed electronics", "discgear", "dp audio video", "dp video", "draper", "eaton", "elago", "electrified", "ematic", "emerge tech", "encore", "endust", "eveready", "fantom", "franklin", "franklin electronics", "gator", "gator cases", "gbc", "ge", "gear head", "general electric", "genius", "genuine phillips", "global marketing", "global marketing partners", "gn netcom", "goodhope bags", "gpx", "grade-a", "greatshield", "green", "griffin technology", "grt", "guardian", "hewcpn", "hon", "hqrp", "human toolz", "humantoolz", "i concepts", "i-tec", "i-tec electronics", "i.sound", "ifrogz", "igo", "imation", "inland", "inland pro", "innergie", "innovera", "iosafe inc", "iris", "iriver", "itw", "itw dymon", "ivina", "jawbone", "jwin", "kanguru", "kanguru solutions", "karendeals", "kenwood", "keyspan", "keytronicems", "kimberly-clark", "kimberly-clark professional", "kinamax", "kingsmart", "kingwin", "kiq", "konica-minolta", "koss", "labtec", "lanthem", "lathem", "lexar", "lexar media", "lifeworks", "lorex", "lumiere l.a.", "lumiere la", "luxor", "macally", "maccase", "mace", "mace security", "mach speed", "magnavox", "manhattan", "master caster", "maxell", "mayline", "mead", "memorex", "memtek", "mercury", "mercury luggage", "merkury", "merkury innovations", "metra", "micro innovations", "micronet", "middle atlantic", "midland", "mionix", "mmf", "mobile edge", "mohawk", "molex", "motion systems", "mukii", "mygear products", "nan", "national", "national products ltd.", "navgear", "neat receipts", "neatreceipts", "next web sales", "nextware", "night owl", "night owl optics", "norazza", "nxg", "nxg technology", "nzxt", "office star", "oki data", "omnimount", "orbital", "oxford", "p3 international", "p3 international corporation", "paper mate", "pc treasures", "peak", "peerless", "pelican", "pelican storm", "pelouze", "pentel of america ltd.", "performance plus", "planar", "planar systems", "plugable technologies", "pm company", "power mat", "power mate", "powermat", "premiertek", "primera technology", "pyle", "pyramid", "q-see", "quality park", "raptor", "raptor-gaming", "rayovac", "read right", "rim", "riteav", "roocase", "royal", "royal consumer", "s j paper", "sabrent", "saitek", "sakar", "sanford", "sanrio", "sanus", "sanyo", "sapphire", "scosche", "scotch", "seal shield", "seiko", "seiko instruments", "sentry", "sgp", "shopforbattery", "simplism japan", "siskiyou", "skooba design", "slappa", "slik", "smartbuy", "smk", "solidtek", "solo", "sonnet technologies", "sonnet technologies inc", "sparco", "spectrum brands", "srs", "srs labs", "stanley", "stanley bostitch", "startech", "startech.com", "startech.com usa llp", "steelseries", "storm", "sumas media", "sumdex", "sunvalleytek", "super talent", "svat", "svat electronics", "swann", "sylvania", "t-mobile", "talk works", "tandberg", "team pro mark", "team promark", "team promark llc", "tektronix", "ten", "ten one design", "terk", "the joy factory", "thermaltake", "tomtom", "tp link", "tracfone", "tvtimedirect", "ultralast", "uniden", "universal products", "us brown bear", "usrobotics", "veho", "verbatim", "victor", "victory", "victory multimedia", "vipertek", "visioneer", "visiontek", "vistablet", "visual land", "vonnic inc", "wacom", "wacom tech corp.", "wausau", "wausau paper", "weyerhauser", "whistler", "whistler radar", "wincraft", "wintec", "x-acto", "x16-81686-03", "xantech", "xfx", "xgear", "xo vision", "xtrememac", "zalman usa inc", "zax", "zebra", "zoom", "zoom telephonics", "zotac", "zune"],
    'UNK':["aiptek", "alera", "aleratec", "alkaline", "allsop", "amp energy", "ampad", "arclyte technologies inc. .", "atrend-bbox", "bell  o", "bravo", "bravo view", "buddy", "can-c", "channel sources", "clover", "clover electronics", "cms", "curo7", "pentel of america ltd. ."]
}
category_hierarchy = {
    "Electronics": {
        "Audio": {
            "Cables": ["audio cables", "speaker cables", "rca cables", "video cables", "cables interconnects", "power cables"],
            "Mobile Phones": ["prepaid wireless phones", "unlocked cell phones", "smartphones", "phones", "feature phones", "corded telephones", "cordless telephones", "phones with plans", "unlocked phones", "no-contract phones"],
            "Car Audio": ["audio car mounts", "car audio video", "car stereos", "bluetooth car kits", "auxiliary input adapters", "audio-video kits", "car kits", "radio antennas", "equalizers"],
            "Speakers": ["stereos/audio", "coaxial speakers", "speakers", "subwoofers", "component subwoofers", "speaker systems", "speaker parts components", "subwoofer boxes and enclosures", "boomboxes"],
            "Accessories": ["amplifier installation", "amplifier wiring kits", "speaker installation", "mp3 accessories", "speaker connectors", "bluetooth headsets", "universal fm cassette adapters", "mp3 player accessories", "mp3 player cables adapters", "headsets", "wired headsets", "headsets microphones", "microphones accessories", "touch screen tablet accessories"],
            "General": ["audio video accessories", "computers accessories", "phone accessories", "electronics-inflexible kit", "satellite radios", "satellite radio", "antennas", "handheld portable satellite radios", "plug play satellite radios", "component video", "stereos", "answering devices", "caller id displays", "corded-cordless combo telephones"]
        },
        "Cameras": {
            "Types": ["digital cameras", "film cameras", "hidden cameras", "game cameras", "bullet cameras", "dome cameras", "simulated cameras", "camcorders", "dslr cameras", "digital slr cameras", "digital slr camera bundles", "point shoot digital cameras", "point shoot digital camera bundles", "film"],
            "Accessories": ["camera batteries", "lens accessories", "tripods", "camera and camcorder accessories", "camera lenses", "binocular accessories", "filters", "camera bags", "camcorder batteries", "photo video design", "digital camera accessories", "flashes", "complete tripod units", "professional video accessories", "webcams", "external floppy drives", "faceplates", "screen filters"],
            "General": ["camera photo", "binoculars"]
        },
        "Computing": {
            "Components": ["motherboards", "graphics cards", "hard drives", "internal hard drives", "external hard drives", "internal optical drives", "internal sound cards", "computer components", "memory", "memory cards", "usb port cards", "network cards", "case fans", "video capture cards", "optical drives", "hard drive enclosures", "firewire port cards", "internal dvd drives", "i o port cards", "floppy diskettes"],
            "Accessories": ["keyboard mouse combos", "mouse pads", "computers", "laptop computers", "monitor arms stands", "computer cases", "cooling pads", "usb cables", "power supplies", "computer accessories", "memory card readers", "memory card adapters", "monitors", "mice", "gaming mice", "modems", "scsi port cards", "computer cable adapters", "computer monitor mounts", "computer speakers", "cases", "keyboards", "laptops", "netbooks", "trackballs", "touch pads", "headphones", "headphone accessories", "desktops", "keyboards mice input devices", "keyboards styluses", "laptop netbook computer accessories", "tablet accessories", "docking stations", "graphics tablets", "monitor accessories", "mounts", "tablets", "ipod"],
            "Software": ["operating systems", "productivity software", "security software", "graphic design software", "development tools", "software"]
        },
        "Networking": {
            "Devices": ["routers", "walkie-talkie/frs", "network adapters", "wireless access points", "modems", "powerline network adapters", "print servers", "switches", "network attached storage", "frs two-way radios", "gmrs-frs two-way radios", "two-way radios accessories", "kvm switches", "usb network adapters", "networking products"],
            "Cables": ["ethernet cables", "usb cables", "data cables", "firewire cables", "scsi cables", "dvi cables", "hdmi cables", "charger cables", "chargers cables"]
        },
        "Televisions": {
            "TVs": ["tvs", "television", "televisions video", "overhead video"],
            "Accessories": ["tv mounts", "projection screens", "television stands entertainment centers", "video projectors", "video projector accessories", "remote controls", "remote-control extenders", "tv accessories", "video glasses", "projector accessories", "projector mounts", "video converters", "video receiving & installation", "video transmission systems", "controllers"]
        },
        "General": ["electronics - general", "networking", "office electronics", "office electronics accessories", "hubs", "components", "power strips", "cleaning repair", "distribution panels", "electronics", "electronics : flat panel tv", "audio-video shelving", "impact dot matrix printer ribbons", "electrical", "distribution", "surge protectors", "uninterrupted power supply ups", "laser printers", "printing"]
    },
    "Office Supplies": {
        "Paper Products": ["laminating supplies", "business cards", "memo scratch pads", "labels stickers", "postcards", "wide-format paper", "business paper products", "colored paper", "file folder labels", "roll paper", "shipping labels", "address labels", "continuous-form labels", "all-purpose labels", "photo paper", "paper"],
        "Furnitures": ["furniture", "desks", "chairs", "file cabinets", "office furniture lighting", "hanging folders interior folders", "footrests", "stands", "wrist rests"],
        "Electronics": ["printers", "scanners", "shredders", "fax machines", "inkjet all-in-one printers", "laser all-in-one printers", "digital security recorders", "all-in-one printers", "projection screens", "desktop staplers", "telephone accessories", "scanner accessories", "label makers", "postal scales"],
        "Printer Accessories": ["printer accessories", "inkjet printer ink", "laser printer toner", "printer ink toner", "printer labels laser inkjet", "printer staples", "printer roll holders", "printer transfer rollers", "printer transfer units", "inkjet printer paper"],
        "Stationery": ["self-stick notes", "lamps", "pen holders", "stationery & office machinery", "labeling tapes", "binder index dividers", "looseleaf binder paper", "labels stickers", "badge holders", "storage presentation materials", "document creation", "view binders", "english dictionaries", "self-stick note pad holders", "flowcharts", "portfolios", "presentation pointers", "presentation remotes", "index dividers"]
    },
    "Automotive": {
        "Electronics": ["dash mounting kits", "radar detectors", "gps", "handheld gps", "vehicle gps", "portable vehicle gps", "in-dash navigation", "audio-video kits", "gps system accessories", "car electronics"],
        "Accessories": ["car chargers", "vehicle mounts", "back seat cushions", "car cradles", "dash mounting kits", "wiring harnesses", "auxiliary input adapters", "cb radios", "antitheft", "car electronics accessories"],
        "General": ["automotive - general"]
    },
    "Batteries": {
        "Types": ["12v", "6v", "9v", "aa", "aaa", "coin button cell", "batteries"],
        "Accessories": ["battery chargers", "charger cables", "chargers adapters", "household batteries", "batteries chargers accessories", "desktop chargers", "ac adapters", "chargers"]
    },
    "Media": {
        "Storage": ["storage", "cd-r discs", "dvd r discs", "usb flash drives", "usb drives", "external data storage", "data cartridges", "dvd rw discs", "bd-r discs", "blank media", "cd-rw discs", "dvd-ram discs", "dlt cleaning cartridges", "media storage organization", "disc jewel cases", "disc storage wallets", "dvd-r discs"],
        "Devices": ["blu-ray disc players", "mp3 players", "portable dvd players", "dvd players", "dvd accessories", "upconverting dvd players", "disc players recorders", "mp3", "portable audio", "cd players", "cd-mp3 players", "digital media devices", "changers"]
    },
    "Home & Security": {
        "Home Theater": ["dvd home theater", "home theater systems", "home theater", "stereo amplifiers", "home audio theater", "home audio crossovers", "turntables"],
        "Security": ["home care", "home security systems", "surveillance cameras", "security surveillance", "security sensors alarms", "complete surveillance systems", "hidden cameras", "household sensors alarms", "simulated cameras", "security monitors displays", "surveillance video recorders"],
        "General": ["house wares"]
    },
    "Accessories": {
        "General": ["bags cases", "covers skins", "armbands", "cases sleeves", "handbags", "camera bags", "accessories", "accessories supplies", "accessories apparel", "accessory kits", "cases bags", "hard drive bags", "projector bags cases", "covers", "armbands", "styli", "screen protectors", "screen protector foils", "wall chargers", "laser pointers"]
    },
    "Tools": {
        "General": ["toolkits", "power strips", "cable security devices", "cleaning repair", "binding machines", "binding machine supplies", "adapters", "connectors adapters", "adapter rings", "distribution panels", "wire management", "power adapters", "power converters", "power ground cable", "power inverters", "power-cable terminals", "selector boxes", "switchers"]
    },
    "Photography": {
        "Cameras": ["point shoot digital cameras", "dslr cameras", "camcorders", "digital slr cameras", "digital slr camera bundles", "point shoot digital camera bundles"],
        "Accessories": ["filters", "camera bags", "tripods", "camera batteries", "camera lenses", "flashes", "lens accessories", "complete tripod units", "professional video accessories"],
        "General": ["photography - general", "photo editing", "media storage organization"]
    },
    "Other": {
        "General": ["scientific", "fan shop", "rugs", "toys - games", "garden - general", "sports outdoor gps", "hiking gps units", "boating gps units chartplotters", "fish finder", "fishfinders", "fishing and boating", "nan", "personal care", "other office equipment", "outlet plates", "wall plates connectors", "radios", "basic", "bundles", "c", "d", "hardware", "lighting", "pda handheld accessories"]
    }
}




def make_mask(df,frac):
    L = len(COL)*len(df)
    L_disturb = int(np.ceil(L*frac))
    disturb_mask = np.zeros((len(df),len(COL)))
    total_elements = disturb_mask.size
    random_indices = np.random.choice(total_elements, L_disturb, replace=False)
    disturb_mask.flat[random_indices] = 1
    return disturb_mask 


def find_parents(hierarchy, target, parents=None):
    if parents is None:
        parents = []

    for key, value in hierarchy.items():
        if key == target:  # If the target itself is a key
            return parents + [key]
        if isinstance(value, dict):  # If value is a nested dictionary
            result = find_parents(value, target, parents + [key])
            if result != -1:
                return result
        elif isinstance(value, list) and target in value:  # If value is a list containing the target
            return parents + [key]

    return -1  # Return -1 if the target is not found


def disturb_func(col,value):
    if col =='brand':
        
        parents = find_parents(brand_hierarchy, str(value).lower().strip())
        if parents == -1: print(value)
        new_seed = int(time.time())
        np.random.seed(new_seed)
        np.random.shuffle(parents)
        new_velue = parents[0]
        

    elif col == 'category':
        parents = find_parents(category_hierarchy, str(value).lower().strip())
        if parents == -1: print(value)
        new_seed = int(time.time())
        np.random.seed(new_seed)
        np.random.shuffle(parents)
        new_velue = parents[0]

    elif col =='price':
        if str(value) =='nan':
            new_velue = 'other'

        else:
                        
            data = [float(value)]
            number_hierarchy = {
                "Ranges": {
                    "Very Low": {
                        "0-0.49": [n for n in data if 0 <= n < 0.5],
                        "0.5-0.99": [n for n in data if 0.5 <= n < 1]
                    },
                    "Low": {
                        "1-4.99": [n for n in data if 1 <= n < 5],
                        "5-9.99": [n for n in data if 5 <= n < 10]
                    },
                    "Moderate": {
                        "10-49.99": [n for n in data if 10 <= n < 50],
                        "50-99.99": [n for n in data if 50 <= n < 100]
                    },
                    "High": {
                        "100-499.99": {
                            "100-199.99": [n for n in data if 100 <= n < 200],
                            "200-299.99": [n for n in data if 200 <= n < 300],
                            "300-399.99": [n for n in data if 300 <= n < 400],
                            "400-499.99": [n for n in data if 400 <= n < 500]
                        },
                        "500-999.99": {
                            "500-749.99": [n for n in data if 500 <= n < 750],
                            "750-999.99": [n for n in data if 750 <= n < 1000]
                        }
                    },
                    "Very High": {
                        "1000-4999.99": {
                            "1000-1999.99": [n for n in data if 1000 <= n < 2000],
                            "2000-2999.99": [n for n in data if 2000 <= n < 3000],
                            "3000-3999.99": [n for n in data if 3000 <= n < 4000],
                            "4000-4999.99": [n for n in data if 4000 <= n < 5000]
                        },
                        "5000 and above": {
                            "5000-9999.99": [n for n in data if 5000 <= n < 10000],
                            "10000 and above": [n for n in data if n >= 10000]
                        }
                    }
                },
                
            }
            if find_parents(number_hierarchy, float(value)) ==-1:
                print(value, type(value))
            parents = find_parents(number_hierarchy, float(value))[1:]

            new_seed = int(time.time())
            np.random.seed(new_seed)
            np.random.shuffle(parents)
            new_velue = parents[0]
        
    return new_velue



# /home/mmoslem3/DATA_VLDB/Fodors-Zagat/

task = 'Walmart-Amazon'
COL = ['category','brand','price']


import os
import random
for frac in [0,0.1,0.2,0.3,0.4,0.5]:
    L =[]
    for rep in range(1,101):
        df = pd.read_csv("/Users/mohammad/Desktop/EM_Heterogeneity/data/" + task  +'/test.csv')

        mask_left = make_mask(df,frac)
        mask_right = make_mask(df,frac)
        df['left_price'] = df['left_price'].astype(str)
        df['right_price'] = df['right_price'].astype(str)
        x = 0
        for i,col in enumerate(COL):
            df.loc[mask_left[:,i] == 1, 'left_'+col] = df.loc[mask_left[:,i] == 1, 'left_'+col].apply(lambda x: disturb_func(col, x))
            df.loc[mask_right[:,i] == 1, 'right_'+col] = df.loc[mask_right[:,i] == 1, 'right_'+col].apply(lambda x: disturb_func(col, x))

            x+=(column_entropy(df['left_'+col]) +column_entropy(df['right_'+col]))

        L.append(x)
    if frac ==0:
        base = np.average(L).copy()
    base =1
    print(round(np.average(L)/base,2), round(np.sqrt(np.std(L))/base,2))

print('walamrt')


39.17 0.0
39.18 0.46
38.04 0.82
36.15 0.7
35.01 0.73
32.66 1.05
walamrt


In [3]:
import time
date_hierarchy = {
    '1980s': {
        '1987': {
            '1987September': ['1987/September/23']
        }
    },
    '1990s': {
        '1990': {
            '1990January': ['1990/January/01']
        },
        '1994':[],
        '1997':[],
        '1995': {
            '1995June': ['1995/June/13']
        },
        '1999': {
            '1999May': ['1999/May/17'],
            '1999October': ['1999/October/19']
        }
    },
    '2000s': {
        '2000': {
            '2000August': ['2000/August/17'],
            '2000September': ['2000/September/26']
        },
        '2002': {
            '2002April': ['2002/April/02']
        },
        '2003': {
            '2003May': ['2003/May/20', '2003/May/27'],
            '2003August': ['2003/August/29'],
            '2003November': ['2003/November/11']
        },
        '2004': {
            '2004February': ['2004/February/02', '2004/February/10'],
            '2004July': ['2004/July/27'],
            '2004November': ['2004/November/16']
        },
        '2005': {
            '2005September': ['2005/September/20'],
            '2005October': ['2005/October/04'],
            '2005November': ['2005/November/07', '2005/November/22']
        },
        '2006': {
            '2006April': ['2006/April/04'],
            '2006June': ['2006/June/20'],
            '2006October':['2006/October/24'],
            '2006March':['2006/March/04']

        },
        '2007': {
             '2007September':['2007/September/18'],
            '2007February': ['2007/February/06'],
            '2007March': ['2007/March/16'],
            '2007July': ['2007/July/10'],
            '2007August': ['2007/August/14'],
            '2007November': ['2007/November/20']
        },
        '2008': {
            '2008March': ['2008/March/17', '2008/March/18'],
            '2008June': ['2008/June/23', '2008/June/24'],
            '2008July': ['2008/July/15', '2008/July/29'],
            '2008August': ['2008/August/01'],
            '2008October': ['2008/October/21', '2008/October/28'],
            '2008November': ['2008/November/04', '2008/November/11']
        },
        '2009': {
            '2009March': ['2009/March/30'],
            '2009April': ['2009/April/17'],
            '2009September':['2009/September/15'],
            '2009May': ['2009/May/05'],
            '2009August': ['2009/August/21'],
            '2009October': ['2009/October/02'],
            '2009November': ['2009/November/03', '2009/November/23'],
            '2009December': ['2009/December/21']
        }
    },










    '2010s': {
        '2010': {
             
            '2010June': ['2010/June/15', '2010/June/16', '2010/June/21'],
            '2010August': ['2010/August/03'],
            '2010October': ['2010/October/25'],
            '2010November': ['2010/November/24', '2010/November/26'],
            '2010December': ['2010/December/14', '2010/December/21','2010/December/03']
        },
        '2011': { 
             



            '2011January': ['2011/January/14'],
            '2011February': ['2011/February/08', '2011/February/14', '2011/February/22'],
            '2011March': ['2011/March/08', '2011/March/28', '2011/March/29'],
            '2011April': ['2011/April/18', '2011/April/26','2011/April/19'],
            '2011May': ['2011/May/23', '2011/May/25', '2011/May/27'],
            '2011June': ['2011/June/07'],
            '2011July': ['2011/July/14', '2011/July/22','2011/July/19'],
            '2011August': ['2011/August/23', '2011/August/26'],
            '2011September': ['2011/September/09', '2011/September/16', '2011/September/30'],
            '2011November': ['2011/November/15', '2011/November/21'],
            '2011December': ['2011/December/06', '2011/December/09', '2011/December/16', '2011/December/20', '2011/December/27','2011/December/12']
        },
        '2012': {
             




            '2012January': ['2012/January/20'],
            '2012February': ['2012/February/22'],
            '2012March': ['2012/March/20', '2012/March/23'],
            '2012April': ['2012/April/24','2012/April/17'],
            '2012May':['2012/May/11'],
            '2012June': ['2012/June/14', '2012/June/19', '2012/June/22'],
            '2012July': ['2012/July/02', '2012/July/31'],
            '2012August': ['2012/August/09', '2012/August/28'],
            '2012September':['2012/September/11'],
            '2012October': ['2012/October/09', '2012/October/30'],
            '2012November': ['2012/November/09', '2012/November/13'],
            '2012December': ['2012/December/04']
        },
        '2013': {
            '2013January': ['2013/January/04', '2013/January/18'],
            '2013February': ['2013/February/05', '2013/February/12','2013/February/27'],
            '2013March': ['2013/March/22', '2013/March/26'],
            '2013April': ['2013/April/02', '2013/April/05', '2013/April/16'],
            '2013May': ['2013/May/28'],
            '2013June': ['2013/June/11'],
            '2013July': ['2013/July/23'],
            '2013August': ['2013/August/06', '2013/August/13', '2013/August/20', '2013/August/27'],
            '2013September': ['2013/September/10', '2013/September/16', '2013/September/24', '2013/September/30'],
            '2013November': ['2013/November/08', '2013/November/11', '2013/November/25'],
            '2013December': ['2013/December/10', '2013/December/16']
        },
        '2014': {
            '2014February': ['2014/February/18', '2014/February/24','2014/February/11'],
            '2014April': ['2014/April/15'],
            '2014May': ['2014/May/06', '2014/May/20', '2014/May/27','2014/May/02'],
            '2014June': ['2014/June/02', '2014/June/04', '2014/June/20', '2014/June/23'],
            '2014July':['2014/July/01','2014/July/07'],
            '2014August': ['2014/August/19'],
            '2014September': ['2014/September/09'],
            '2014October': ['2014/October/14', '2014/October/21', '2014/October/27'],
            '2014November': ['2014/November/04', '2014/November/10', '2014/November/24'],
            '2014December': ['2014/December/23']








        },
        '2015': {
            '2015January': ['2015/January/09', '2015/January/13', '2015/January/21', '2015/January/30'],
            '2015February': ['2015/February/13', '2015/February/24'],
            '2015March': ['2015/March/03', '2015/March/10'],
            '2015April': ['2015/April/06', '2015/April/10', '2015/April/17', '2015/April/21'],
            '2015May': ['2015/May/12', '2015/May/15', '2015/May/18'],
            '2015June': ['2015/June/01', '2015/June/03', '2015/June/09', '2015/June/16', '2015/June/22', '2015/June/23', '2015/June/30'],
            '2015July': ['2015/July/01', '2015/July/03', '2015/July/10'],
            '2015August': ['2015/August/07', '2015/August/10', '2015/August/28', '2015/August/31'],
            '2015September': ['2015/September/04', '2015/September/11', '2015/September/18', '2015/September/21', '2015/September/25'],
            '2015October': ['2015/October/09']
        }
    }
}











genre_hierarchy = {
    "alternative rock": [
        "alternative rock",
        "alternative rock, indie & lo-fi",
        "alternative, music, rock, adult alternative"
    ],
    "country": [
        "country",
        "country, contemporary country",
        "country, country rock, pop, rock",
        "country, country rock, rock",
        "country, traditional country, cowboy",
        "country, contemporary country, honky tonk",
        "country, honky tonk, contemporary country",
        "country, honky tonk, pop, pop/rock, contemporary country, rock",
        "country, honky tonk, urban cowboy, contemporary country",
        "country, music, pop, teen pop, honky tonk, contemporary country",
        "country, music, rock, contemporary country, pop, pop/rock, honky tonk",
        "country, music, urban cowboy, contemporary country",
        "country, music, urban cowboy, honky tonk, contemporary country",
        "country, music, contemporary country",
        "country, music, contemporary country, honky tonk, traditional country",


                'country, music, honky tonk, contemporary country',
                'country, music, honky tonk, pop, pop/rock, contemporary country, rock',
                'country, music, honky tonk, urban cowboy, contemporary country'
                

    ],
    "dance & electronic": [
        "dance & electronic",
        "dance & electronic, dubstep",
        "dance & electronic, house",
        "dance & electronic, pop, house",
        "dance & electronic, pop, r&b",
        "dance & electronic, rap & hip-hop, house",
        "dance, music",
        "dance, music, electronic",
        "dance, music, electronic, classical, classical crossover, rock, house, electronica",
        "dance, music, electronic, house, rock, french pop",
        "dance, music, pop",
        "dance, music, r&b / soul, electronic",
        "dance, music, rock",
        "dance, music, rock, electronic",
        "dance, music, rock, house, electronic",
        "dance, music, rock, house, electronic, french pop",
        "dance, music, rock, pop, house, electronic, electronica, adult alternative",
        "dance, music, hip-hop / rap, alternative rap, hip-hop, r&b / soul, soul, electronic",
        "dance, music, hip-hop / rap, dirty south, rap, electronic",
        'dance,music,hip-hop / rap, alternative rap,hip-hop, r&b / soul, soul, electronic',
'dance,music,hip-hop / rap, dirty south, rap, electronic',
    ],
    "electronica": [
        "electronica, dance & electronic",
        "electronica, dance & electronic, dubstep",
        "electronica, dance & electronic, house",
        "electronic, music",
        "electronic, music, dance, rock, electronica",
'electronic,music,hip-hop / rap, rap',

    ],
    "folk": [
        "folk, rock"
    ],
    "gangsta & hardcore": [
        "gangsta & hardcore, rap & hip-hop"
    ],
    "gospel": [
        "gospel, christian"
    ],
    "holiday": [
        "holiday, christmas, miscellaneous"
    ],
    "international": [
        "international",
        "international, latin music",
        "international, latin music, latin hip-hop"
    ],
    "miscellaneous": [
        "miscellaneous",
        'house, music, dance, rock, electronic',

    ],
    "pop": [
        "pop",
        "r&b, pop",
        "pop, music, electronic, r&b / soul, pop/rock, dance",
        "pop, music, r&b / soul, contemporary r&b, hip-hop/rap, pop/rock, rock",
        "pop, music, r&b / soul, dance, teen pop, rock",
        "pop, music, r&b / soul, rock, dance, contemporary r&b",
        "pop, music, r&b / soul, teen pop, dance, rock",
        "pop, music, r&b / soul, soul, dance, rock, jazz, hip-hop / rap, electronic, hip-hop, pop/rock, adult alternative",
        "pop, music, rock",
        "pop, music, rock, pop/rock, teen pop, dance",
        "pop, music, rock, r&b / soul, contemporary r&b, dance",
        "pop, music, rock, r&b / soul, contemporary r&b, dance, electronic, hip-hop / rap, pop/rock",
        "pop, music, rock, singer/songwriter, contemporary singer/songwriter, adult alternative",
        "pop, music, r&b / soul, soul, dance, rock, jazz, hip-hop / rap, electronic, hip-hop, pop/rock, adult alternative",
        'pop, music',
'pop, music, r&b / soul,soul,dance,rock,jazz,hip-hop / rap,electronic,hip-hop, pop/rock, adult alternative',
'pop, music, rock, r&b / soul, contemporary r&b, dance,electronic,hip-hop / rap, pop/rock',
    ],
    "r&b": [
        "r&b"
    ],
    "rap & hip-hop": [
        "rap & hip-hop",
        "rap & hip-hop, southern rap, pop rap",
        "rap & hip-hop, west coast",
        "hip-hop/rap, music",
        "hip-hop/rap, music, dirty south",
        "hip-hop/rap, music, east coast rap, hardcore rap, rap",
        "hip-hop/rap, music, hardcore rap, east coast rap, rap",
        "hip-hop/rap, music, pop, dirty south",
        "hip-hop/rap, music, r&b / soul, contemporary r&b, dance, rap",
        "hip-hop/rap, music, rap",
        "hip-hop/rap, music, rap, dirty south",
        "hip-hop/rap, music, rap, east coast rap, hardcore rap",
        "hip-hop/rap, music, rap, electronic, world, dance",
        "hip-hop/rap, music, rap, hardcore rap, east coast rap, rock",
        "hip-hop/rap, music, rock, gangsta rap, west coast rap",
        "dance, music, hip-hop / rap, alternative rap, hip-hop, r&b / soul, soul, electronic",
        "dance, music, hip-hop / rap, dirty south, rap, electronic"
    ],
    "rock": [
        "rock",
        "rock, music, hard rock, alternative",
        "rock, music, metal, alternative, hard rock",
        "singer/songwriter, music, rock",
        'rock, music'
    ],
    "soundtracks": [
        "soundtracks",
        "soundtrack, music, soundtrack, classical, original score",
        "soundtrack, music, hip-hop / rap, west coast rap, gangsta rap, hardcore rap, rap, soundtrack",
        'soundtrack,music,hip-hop / rap, west coast rap, gangsta rap, hardcore rap, rap, soundtrack'

    ]
}


price_hierarchy = {
    "tier one": [
        '$ 0.69',
        '$ 0.89'
    ],
    "tier two": [
        "$ 0.99",'$ 1.29','$ 1.99'
    ],}


def make_mask(df,frac):
    L = len(COL)*len(df)
    L_disturb = int(np.ceil(L*frac))
    disturb_mask = np.zeros((len(df),len(COL)))
    total_elements = disturb_mask.size
    random_indices = np.random.choice(total_elements, L_disturb, replace=False)
    disturb_mask.flat[random_indices] = 1
    return disturb_mask 


def find_parents(hierarchy, target, parents=None):
    if parents is None:
        parents = []

    for key, value in hierarchy.items():
        if key == target:  # If the target itself is a key
            return parents + [key]
        if isinstance(value, dict):  # If value is a nested dictionary
            result = find_parents(value, target, parents + [key])
            if result != -1:
                return result
        elif isinstance(value, list) and target in value:  # If value is a list containing the target
            return parents + [key]

    return -1  # Return -1 if the target is not found


def convert_to_seconds(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds


def disturb_func(col,value):
    if col =='Price':
        if value == 'Album Only':
            new_velue = 'other'
        else:
            parents = find_parents(price_hierarchy, value)
            if parents == -1: print(value)
            new_seed = int(time.time())
            np.random.seed(new_seed)
            np.random.shuffle(parents)
            new_velue = parents[0]
        

    elif col == 'Genre':
        parents = find_parents(genre_hierarchy,value.lower().replace(' , ',', '))
        if parents == -1: print(value)
        new_seed = int(time.time())
        np.random.seed(new_seed)
        np.random.shuffle(parents)
        new_velue = parents[0]


    elif col == 'Released':
        # print(value)



        if str(value) =='nan':
            new_velue = 'other'            

        elif '-' in value:
            value = value.split('-')
            
            if len(value[0]) ==1:
                value[0] = '0'+str(value[0])

            if int(value[2]) > 20:
                yy = '19'+str(value[2])
            else:
                yy = '20'+str(value[2])
                tr_dict = {'Jan':'January',
                        'Feb':'February',
                        'Mar':'March',
                        'Apr':'April',
                        'May':'May',
                        'Jun':'June',
                        'Jul':'July',
                        'Aug':'August',
                        'Sep':'September',
                        'Oct':'October',
                        'Nov':'November',
                        'Dec':'December'}

            value =  yy + '/'   + tr_dict[value[1]]+'/'+ value[0]
            parents = find_parents(date_hierarchy, value)

        elif ',' in value:
            value = value.replace(', ','').split(' ')
            
            if len(value[1]) ==1:
                value[1] = '0'+value[1]

                    
            value= value[2]+'/'+value[0]+'/'+value[1]
            parents = find_parents(date_hierarchy, value)

        else:
            parents = find_parents(date_hierarchy, value)
        if str(value) !='nan':
            # print(parents,value)
            new_seed = int(time.time())
            np.random.seed(new_seed)
            np.random.shuffle(parents)
            new_velue = parents[0]

    elif col =='Time':
        if str(value) =='nan':
            new_velue = 'other'

        else:

            data_in_seconds = [convert_to_seconds(length) for length in [value]]
            music_hierarchy = {
                "Short": {
                    "Very Short": [n for n in data_in_seconds if n < 60],
                    "Short high": [n for n in data_in_seconds if 60 <= n < 180]
                },
                "Moderate": {
                    "Moderate Low": [n for n in data_in_seconds if 180 <= n < 300],
                    "Moderate High": [n for n in data_in_seconds if 300 <= n < 480]
                },
                "Long": {
                    "Long Low": [n for n in data_in_seconds if 480 <= n < 600],
                    "Long High": [n for n in data_in_seconds if 600 <= n < 900]
                },
                "Very Long": {
                    "Very Long Low": [n for n in data_in_seconds if 900 <= n < 1200],
                    "Very Long High": [n for n in data_in_seconds if n >= 1200]
                }
            }
            parents = find_parents(music_hierarchy, data_in_seconds[0])
            if parents == -1:
                print(value,'time')
            new_seed = int(time.time())
            np.random.seed(new_seed)
            np.random.shuffle(parents)
            new_velue = parents[0]
        
    return new_velue




task = 'iTunes-Amazon'
COL = ['Genre', 'Price', 'Time','Released']


import os
import random
for frac in [0,0.1,0.2,0.3,0.4,0.5,0.7,0.9]:
    L =[]
    for rep in range(1,101):
        df = pd.read_csv("/Users/mohammad/Desktop/EM_Heterogeneity/data/" + task  +'/test.csv')

        mask_left = make_mask(df,frac)
        mask_right = make_mask(df,frac)
        x = 0

        for i,col in enumerate(COL):
            df.loc[mask_left[:,i] == 1, 'left_'+col] = df.loc[mask_left[:,i] == 1, 'left_'+col].apply(lambda x: disturb_func(col, x))
            df.loc[mask_right[:,i] == 1, 'right_'+col] = df.loc[mask_right[:,i] == 1, 'right_'+col].apply(lambda x: disturb_func(col, x))

            x+=(column_entropy(df['left_'+col]) +column_entropy(df['right_'+col]))

        L.append(x)
    if frac ==0:
        base = np.average(L).copy()
    base =1
    print(round(np.average(L)/base,2), round(np.sqrt(np.std(L))/base,2))

print('itun')


32.84 0.0
33.98 0.23
34.23 0.75
33.89 0.0
31.82 0.47
29.15 1.09
21.33 0.66
17.9 0.0
itun
